In [ ]:
%pip install pycocotools #our annotations are in coco json


In [ ]:
# helpful sources for COCO: https://www.v7labs.com/blog/coco-dataset-guide
# https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/md-coco-overview.html
# https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocotools/coco.py
from pycocotools.coco import COCO
import os
annotation_file = '../Annotations/combined_annotations.coco.json'
coco = COCO(annotation_file)
#now I have to make sure I get the ID for each and every image
image_ids = coco.getImgIds()
images = coco.loadImgs(image_ids)

#preping image data for keras:
for img in images: #iterating over each image in list of images from COCO dataset
    image_id = img['id'] # 'id' is the unique image identifier
    file_name = img['file_name'] #image's file name
    
    annotation_ids = coco.getAnnIds(imgIds=image_id) #this gets the annotation ID"s associated with the given image in the for loop
    the_annotations = coco.loadAnns(annotation_ids)  #loadAnns retrieves annotation data as a LIST of dictionaries
        #each dict includes data on the image like bounding box (bbox)
        
    bboxes = [ann['bbox'] for ann in the_annotations]
    
    #print(bboxes)  ok we have a lot of different numbers here
    

In [ ]:
#now I just want to check the total number of images we have
import os
image_dir = '../Image_data/DS 4002 Project 3 raw image data'
# get a simple list of all files
all_files = os.listdir(image_dir)
# Filter out non-image files incase i skrewed something up
image_files = [f for f in all_files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

num_images = len(image_files) #this gives the total count of images we have
print(f'Total number of images: {num_images}')
print("great, we have 1601 images in our repo!")



In [ ]:
#ok, now I'll split our data into an 20/80 train/test split using sklearn
import sklearn.model_selection
from sklearn.model_selection import train_test_split
train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)
print(f'Number of training images: {len(train_files)}')
print(f'Number of validation images: {len(val_files)}')

In [ ]:


#next I'm going to use cv2, which is helpful for doing things like image resizing
#source: https://konfuzio.com/en/cv2/#:~:text=The%20cv2%20module%20is%20the,commonly%20used%20functions%20in%20cv2.
%pip install opencv-python-headless
import cv2
import numpy as np
import matplotlib.pyplot as plt
#I will start by initializing each images' width and height in a list
widths = []
heights = []

for file in image_files:
    image_path = os.path.join(image_dir, file) #make sure paths are consistent across systems, in case this is replicated elsewhere
        #again, this is image_dir: image_dir = '../Image_data/DS 4002 Project 3 raw image data'
    image = cv2.imread(image_path) 
        #print(image) --> did this to check, this is a 3 element list of info abt each image
    h, w = image.shape[:2]  #for each image we get height and width, the :2 is to extract only these first two elements
    widths.append(w)  # Add width to the list
    heights.append(h)  # Add height to the list

widths = np.array(widths)
heights = np.array(heights)


